<a href="https://colab.research.google.com/github/doublelockcup/Pathology-Images-Classification/blob/master/HW5_Tianbo_Qiu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow-gpu==2.0.0-alpha0
!pip install sacrebleu # https://github.com/mjpost/sacreBLEU

    100% |████████████████████████████████| 332.1MB 48kB/s 
    100% |████████████████████████████████| 3.0MB 8.7MB/s 
    100% |████████████████████████████████| 61kB 25.6MB/s 
    100% |████████████████████████████████| 419kB 4.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/56/c0/fb/1c7f9b3a71f64cdf86291cc645596f71746807bf2f72b3c1dd
Successfully built sacrebleu


In [2]:
import numpy as np
import re
import sacrebleu
import tensorflow as tf
import time
import unicodedata
print(tf.__version__)

2.0.0-alpha0


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cp '/content/gdrive/My Drive/DeepLearning/datasets/spa.txt' 'spa.txt'

#Part 1

Translate English to Spanish

In [0]:
spa = open('spa.txt', 'r')

In [6]:
data = spa.read().splitlines()
sentences_data = [tuple(d.split('\t')) for d in data]

print(len(sentences_data))
print(sentences_data[:5])

118964
[('Go.', 'Ve.'), ('Go.', 'Vete.'), ('Go.', 'Vaya.'), ('Go.', 'Váyase.'), ('Hi.', 'Hola.')]


In [9]:
def preprocess(s):
  # for details, see https://www.tensorflow.org/alpha/tutorials/sequences/nmt_with_attention
  s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
  s = re.sub(r"([?.!,¿])", r" \1 ", s)
  s = re.sub(r'[" "]+', " ", s)
  s = re.sub(r"[^a-zA-Z?.!,¿]+", " ", s)
  s = s.strip()
  s = '<start> ' + s + ' <end>'
  return s

# sample 3000 sentences as training set
size = 3000
sentences = [(preprocess(source), preprocess(target)) for (source, target) in sentences_data[:size]]
print('Original:', sentences_data[1500])
print('Preprocessed:', sentences[1500])

source_sentences, target_sentences = list(zip(*sentences))

Original: ('We must go.', 'Nos debemos ir.')
Preprocessed: ('<start> We must go . <end>', '<start> Nos debemos ir . <end>')


In [0]:
class Translator:
  def __init__(self, source_sentences, target_sentences):
    self.source_sentences = source_sentences
    self.target_sentences = target_sentences
    self.source_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    self.source_tokenizer.fit_on_texts(source_sentences)
    source_data = self.source_tokenizer.texts_to_sequences(source_sentences)
    self.source_data = tf.keras.preprocessing.sequence.pad_sequences(source_data, padding='post')
    
    self.target_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    self.target_tokenizer.fit_on_texts(target_sentences)
    target_data = self.target_tokenizer.texts_to_sequences(target_sentences)
    self.target_data = tf.keras.preprocessing.sequence.pad_sequences(target_data, padding='post')
    
    self.target_labels = np.zeros(self.target_data.shape)
    self.target_labels[:, 0:self.target_data.shape[1]-1] = self.target_data[:,1:]
    
    self.source_vocab_size = len(self.source_tokenizer.word_index) + 1
    self.target_vocab_size = len(self.target_tokenizer.word_index) + 1
    self.batch_size = batch_size= 10
    self.dataset = tf.data.Dataset.from_tensor_slices((self.source_data, self.target_data, self.target_labels)).batch(batch_size)
    
    

  def decode(self, encoded_data):
    for number in encoded_data:
      if number != 0:
        print("%d -> %s" % (number, self.source_tokenizer.index_word[number]))
        
class Encoder(tf.keras.Model):
  def __init__(self, translator):
    super(Encoder, self).__init__()
    self.embedding_size = 32
    self.rnn_size = 64
    
    self.embedding = tf.keras.layers.Embedding(translator.source_vocab_size, self.embedding_size)
    self.gru = tf.keras.layers.GRU(self.rnn_size, return_sequences=True, return_state=True)
  
  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)
    return output, state
  
  def init_state(self, batch_size):
    return tf.zeros((batch_size, self.rnn_size))

class Decoder(tf.keras.Model):
  def __init__(self, translator):
    super(Decoder, self).__init__()
    self.embedding_size = 32
    self.rnn_size = 64
    self.embedding = tf.keras.layers.Embedding(translator.target_vocab_size, self.embedding_size)
    self.gru = tf.keras.layers.GRU(self.rnn_size, return_sequences=True, return_state=True)
    self.dense = tf.keras.layers.Dense(translator.target_vocab_size)
    
  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)
    logits = self.dense(output)
    return logits, state
  

def calc_loss(targets, logits):
  mask = tf.math.logical_not(tf.math.equal(targets, 0))
  mask = tf.cast(mask, dtype=tf.int64)
  crossentropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  return crossentropy(targets, logits, sample_weight=mask)



class Trans:
  def __init__(self, source_sentences, target_sentences):
    self.t = t = Translator(source_sentences, target_sentences)
    self.encoder = Encoder(t)
    self.decoder = Decoder(t)
    self.optimizer = tf.keras.optimizers.Adam()
    self.batch_size= batch_size = t.batch_size
    
  def translate(self, idx=None):
    encoder = self.encoder
    decoder = self.decoder
    source_sentences = self.t.source_sentences
    target_sentences = self.t.target_sentences
    source_data = self.t.source_data
    target_data = self.t.target_data
    target_labels = self.t.target_labels
    target_tokenizer = self.t.target_tokenizer
    #sentences = list(zip(source_sentences, target_sentences))
    if idx == None: 
      idx = np.random.choice(len(source_sentences))

    input_sent = source_data[idx]
    input_sent = tf.expand_dims(input_sent, axis=0)

    hidden_state = encoder.init_state(batch_size=1)
    output, hidden_state = encoder(input_sent, hidden_state)

    decoder_input = tf.expand_dims([target_tokenizer.word_index['<start>']], 0)
    out_words = []

    decoder_state = hidden_state

    while True:

        decoder_output, decoder_state = decoder(decoder_input, decoder_state)
        decoder_input = tf.argmax(decoder_output, -1)
        word_idx = decoder_input.numpy()[0][0]
        # if we've predicted 0 (which is reserved, usually this will only happen
        # before the decoder is trained, just stop translating and return
        # what we have)
        if word_idx == 0: 
          out_words.append('<end>')
        else:
          out_words.append(target_tokenizer.index_word[word_idx])

        if out_words[-1] == '<end>' or len(out_words) >= 20:
          break

    translation = ' '.join(out_words)    
    return source_sentences[idx], target_sentences[idx], translation
  
  
  @tf.function
  def train_step(self, source_seq, target_seq, target_labels, initial_state):
    with tf.GradientTape() as tape:
      encoder_output, encoder_state = self.encoder(source_seq, initial_state)
      logits, decoder_state = self.decoder(target_seq, encoder_state)
      loss = calc_loss(target_labels, logits)
    variables = self.encoder.trainable_variables + self.decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    self.optimizer.apply_gradients(zip(gradients, variables))
    return loss
  
  def train(self, EPOCHS=200):
    batch_size = self.batch_size
    dataset = self.t.dataset
    for epoch in range(EPOCHS):
      start = time.time()
      en_initial_states = self.encoder.init_state(batch_size)
      for batch, (source_seq, target_seq, target_labels) in enumerate(dataset):
        loss = self.train_step(source_seq, target_seq, target_labels, en_initial_states)
        elapsed = time.time() - start
      
      if epoch % 10 == 0:
        print("Epoch #%d, Loss %.4f, Time %.2f sec" % (epoch, loss, elapsed))
        input_sent, target_sent, translation = self.translate()
        print("Input: %s\nTarget: %s\nTranslation: %s\n" % (input_sent, target_sent, translation))
    

In [0]:
trans = Trans(source_sentences, target_sentences)

In [99]:
# before trainings
trans.translate()

('<start> Can you read ? <end>',
 '<start> ¿ Puedes leer ? <end>',
 'mayores deje mayores ricos carne olvidese llevar fantastico chino comprometida equivocado pensando pruebalo viejos resbalo ganaste herido veni ninguno sentaba')

In [100]:
trans.train(EPOCHS=30)

Epoch #0, Loss 1.8154, Time 3.95 sec
Input: <start> Who died ? <end>
Target: <start> ¿ Quien murio ? <end>
Translation: . <end>

Epoch #10, Loss 0.9399, Time 2.15 sec
Input: <start> I am sick . <end>
Target: <start> Estoy enferma . <end>
Translation: me gusta el la perro . <end>

Epoch #20, Loss 0.6965, Time 1.81 sec
Input: <start> I warned Tom . <end>
Target: <start> He avisado a Tom . <end>
Translation: me gusta el tom . <end>



**BLEU score**

In [0]:
def eval(trans):
  references, hypotheses = [], []
  source_sentences = trans.t.source_sentences
  for i in range(len(source_sentences)):
    input_sent, target_sent, translation = trans.translate(i)
    references.append(target_sent)
    hypotheses.append('<start>' + translation)
  results = sacrebleu.raw_corpus_bleu(hypotheses, [references])
  print(results)

In [103]:
eval(trans)

BLEU(score=3.721688841899096, counts=[5996, 2708, 70, 25], totals=[15033, 12033, 9033, 6033], precisions=[39.885585046231625, 22.504778525720933, 0.7749363445145577, 0.4143875352229405], bp=0.9032317693094347, sys_len=15033, ref_len=16563)


#Part 2

Translate from Spanish to Engish

In [104]:
trans2 = Trans(target_sentences, source_sentences)
trans2.train(EPOCHS=20)

Epoch #0, Loss 2.2373, Time 3.80 sec
Input: <start> Esto es una cancion . <end>
Target: <start> It s a song . <end>
Translation: i i . <end>

Epoch #10, Loss 1.0042, Time 1.40 sec
Input: <start> Estamos en casa . <end>
Target: <start> We re home . <end>
Translation: you re a dog . <end>



In [105]:
eval(trans2)

BLEU(score=3.396092637509502, counts=[6218, 2673, 65, 26], totals=[16700, 13700, 10700, 7700], precisions=[37.23353293413174, 19.51094890510949, 0.6074766355140186, 0.33766233766233766], bp=0.9720157863692712, sys_len=16700, ref_len=17174)


#Part 3

Back-translate

In [0]:
class BackTrans:
  def __init__(self, trans, trans2):
    self.en2sp = trans
    self.sp2en = trans2

In [106]:
len(sentences)

3000

In [51]:
t1 = Translator(source_sentences, target_sentences)
print(t1.source_data[1000])
print(t1.target_data[1000])

print(t1.target_labels[200])
print(t1.source_vocab_size)
print(t1.target_vocab_size)

t1.decode(t1.source_data[1000])

ex_sentence = tf.expand_dims(t1.source_data[1000], axis=0)
ex_translation = tf.expand_dims(t1.target_data[1000], axis=0)
ex_labels = tf.expand_dims(t1.target_labels[1000], axis=0)
print(ex_sentence)
print(ex_translation)
print(ex_labels)

encoder = Encoder(t1)
hidden_state = encoder.init_state(1)
print(hidden_state.shape)
output, hidden_state = encoder(ex_sentence, hidden_state)
print(output.shape)

decoder = Decoder(t1)
decoder_output, decoder_state = decoder(ex_labels, hidden_state)
print(decoder_output.shape)

print("Loss", calc_loss(ex_labels, decoder_output))

[  1  13  12  17 432   3   2   0]
[  1  11   9 505   3   2   0   0   0   0   0]
[810.   3.   2.   0.   0.   0.   0.   0.   0.   0.   0.]
918
1909
1 -> <start>
13 -> he
12 -> is
17 -> a
432 -> dj
3 -> .
2 -> <end>
tf.Tensor([[  1  13  12  17 432   3   2   0]], shape=(1, 8), dtype=int32)
tf.Tensor([[  1  11   9 505   3   2   0   0   0   0   0]], shape=(1, 11), dtype=int32)
tf.Tensor([[ 11.   9. 505.   3.   2.   0.   0.   0.   0.   0.   0.]], shape=(1, 11), dtype=float64)
(1, 64)
(1, 8, 64)
(1, 11, 1909)
Loss tf.Tensor(3.4339483, shape=(), dtype=float32)


In [0]:
def translate(idx=None):
  encoder = trans.encoder
  decoder = trans.decoder
  source_data = trans.t.source_data
  target_data = trans.t.target_data
  target_labels = trans.t.target_labels
  target_tokenizer = trans.t.target_tokenizer
  if idx == None: 
    idx = np.random.choice(len(sentences))

  input_sent = source_data[idx]
  input_sent = tf.expand_dims(input_sent, axis=0)

  hidden_state = encoder.init_state(batch_size=1)
  output, hidden_state = encoder(input_sent, hidden_state)

  decoder_input = tf.expand_dims([target_tokenizer.word_index['<start>']], 0)
  out_words = []

  decoder_state = hidden_state

  while True:

      decoder_output, decoder_state = decoder(decoder_input, decoder_state)
      decoder_input = tf.argmax(decoder_output, -1)
      word_idx = decoder_input.numpy()[0][0]
      # if we've predicted 0 (which is reserved, usually this will only happen
      # before the decoder is trained, just stop translating and return
      # what we have)
      if word_idx == 0: 
        out_words.append('<end>')
      else:
        out_words.append(target_tokenizer.index_word[word_idx])

      if out_words[-1] == '<end>' or len(out_words) >= 20:
        break

  translation = ' '.join(out_words)    
  return sentences[idx][0], sentences[idx][1], translation

In [82]:
input_sent, target_sent, translation = translate()
print("Input: %s\nTarget: %s\nTranslation: %s\n" % (input_sent, target_sent, translation))

Input: <start> Wait up . <end>
Target: <start> Esperame despierto . <end>
Translation: esperame despierto . <end>

